In [32]:
import pandas as pd 
import random
from sklearn import model_selection, linear_model

In [9]:
old_df = pd.read_csv("tennis_data.csv")

/var/folders/n4/mqnt6n7d4h50q8s_q29m39w80000gn/T/ipykernel_3324/2298426289.py:1: DtypeWarning: Columns (36) have mixed types. Specify dtype option on import or set low_memory=False.
  old_df = pd.read_csv("tennis_data.csv")


In [10]:
old_df.head()

,ATP,Location,Tournament,Date,Series,Court,Surface,Round,Best of,Winner,...,pl1_flag,pl1_year_pro,pl1_weight,pl1_height,pl1_hand,pl2_flag,pl2_year_pro,pl2_weight,pl2_height,pl2_hand
0,1,Adelaide,Adelaide International 1,2022-01-03,ATP250,Outdoor,Hard,1st Round,3,Kwon S.W.,...,KOR,2015.0,72.0,180.0,Right-Handed,JPN,2014.0,64.0,170.0,Left-Handed
1,1,Adelaide,Adelaide International 1,2022-01-03,ATP250,Outdoor,Hard,1st Round,3,Monteiro T.,...,BRA,2011.0,78.0,183.0,Left-Handed,GER,2014.0,80.0,188.0,Right-Handed
2,1,Adelaide,Adelaide International 1,2022-01-03,ATP250,Outdoor,Hard,1st Round,3,Djere L.,...,SRB,2013.0,80.0,185.0,Right-Handed,ESP,2011.0,76.0,180.0,Right-Handed
3,1,Adelaide,Adelaide International 1,2022-01-03,ATP250,Outdoor,Hard,1st Round,3,Johnson S.,...,USA,2012.0,86.0,188.0,Right-Handed,AUS,2018.0,85.0,188.0,Right-Handed
4,1,Adelaide,Adelaide International 1,2022-01-04,ATP250,Outdoor,Hard,1st Round,3,Moutet C.,...,FRA,2016.0,71.0,175.0,Left-Handed,DEN,2020.0,77.0,188.0,Right-Handed


In [11]:
old_df.columns

Index(['ATP', 'Location', 'Tournament', 'Date', 'Series', 'Court', 'Surface',
       'Round', 'Best of', 'Winner', 'Loser', 'WRank', 'LRank', 'WPts', 'LPts',
       'W1', 'L1', 'W2', 'L2', 'W3', 'L3', 'W4', 'L4', 'W5', 'L5', 'Wsets',
       'Lsets', 'Comment', 'B365W', 'B365L', 'PSW', 'PSL', 'MaxW', 'MaxL',
       'AvgW', 'AvgL', 'EXW', 'EXL', 'LBW', 'LBL', 'SJW', 'SJL', 'UBW', 'UBL',
       'pl1_flag', 'pl1_year_pro', 'pl1_weight', 'pl1_height', 'pl1_hand',
       'pl2_flag', 'pl2_year_pro', 'pl2_weight', 'pl2_height', 'pl2_hand'],
      dtype='object')

In [30]:
# Make new DF for model building
df = pd.DataFrame()
random.seed(42)
for index, row in old_df.iterrows():
    coin = random.random() < 0.5
    new_row = {}
    if coin:
        new_row["P1"] = row["Winner"]
        new_row["P2"] = row["Loser"]
        new_row["P1Rank"] = row["WRank"]
        new_row["P2Rank"] = row["LRank"]
        new_row["B365P1"] = row["B365W"]
        new_row["B365P2"] = row["B365L"]
        new_row["PSP1"] = row["PSW"]
        new_row["PSP2"] = row["PSL"]
        new_row["MaxP1"] = row["MaxW"]
        new_row["MaxP2"] = row["MaxL"]
        new_row["AvgP1"] = row["AvgW"]
        new_row["AvgP2"] = row["AvgL"]
        new_row["P1_year_pro"] = row["pl1_year_pro"]
        new_row["P1_weight"] = row["pl1_weight"]
        new_row["P1_height"] = row["pl1_height"]
        new_row["P2_year_pro"] = row["pl2_year_pro"]
        new_row["P2_weight"] = row["pl2_weight"]
        new_row["P2_height"] = row["pl2_height"]
        new_row["Winner"] = 1
    else:
        new_row["P2"] = row["Winner"]
        new_row["P1"] = row["Loser"]
        new_row["P2Rank"] = row["WRank"]
        new_row["P1Rank"] = row["LRank"]
        new_row["B365P2"] = row["B365W"]
        new_row["B365P1"] = row["B365L"]
        new_row["PSP2"] = row["PSW"]
        new_row["PSP1"] = row["PSL"]
        new_row["MaxP2"] = row["MaxW"]
        new_row["MaxP1"] = row["MaxL"]
        new_row["AvgP2"] = row["AvgW"]
        new_row["AvgP1"] = row["AvgL"]
        new_row["P2_year_pro"] = row["pl1_year_pro"]
        new_row["P2_weight"] = row["pl1_weight"]
        new_row["P2_height"] = row["pl1_height"]
        new_row["P1_year_pro"] = row["pl2_year_pro"]
        new_row["P1_weight"] = row["pl2_weight"]
        new_row["P1_height"] = row["pl2_height"]
        new_row["Winner"] = 0
    new_df = pd.DataFrame([new_row])
    df = pd.concat([df, new_df], axis=0, ignore_index=True)

In [31]:
df.head()

,P2,P1,P2Rank,P1Rank,B365P2,B365P1,PSP2,PSP1,MaxP2,MaxP1,AvgP2,AvgP1,P2_year_pro,P2_weight,P2_height,P1_year_pro,P1_weight,P1_height,Winner
0,Kwon S.W.,Nishioka Y.,53.0,81.0,1.61,2.30,1.70,2.26,1.76,2.50,1.64,2.22,2015.0,72.0,180.0,2014.0,64.0,170.0,0
1,Altmaier D.,Monteiro T.,84.0,89.0,1.61,2.30,1.60,2.49,1.62,2.60,1.57,2.38,2014.0,80.0,188.0,2011.0,78.0,183.0,1
2,Carballes Baena R.,Djere L.,79.0,52.0,2.20,1.66,2.10,1.81,2.26,1.83,2.12,1.70,2011.0,76.0,180.0,2013.0,80.0,185.0,1
3,Vukic A.,Johnson S.,156.0,85.0,1.80,2.00,1.74,2.21,1.80,2.30,1.72,2.10,2018.0,85.0,188.0,2012.0,86.0,188.0,1
4,Moutet C.,Rune H.,92.0,103.0,2.75,1.44,2.66,1.53,2.81,1.54,2.64,1.47,2016.0,71.0,175.0,2020.0,77.0,188.0,0


In [33]:
# Seperate train and test
train_df, test_df = model_selection.train_test_split(df, shuffle=False, test_size = 0.3)